In [1]:
import torch
torch.cuda.empty_cache()
torch.cuda.ipc_collect()


In [2]:
import platform
print(platform.system()) 

Linux


In [3]:
import torch

# PyTorch의 CUDA 버전 확인
if torch.cuda.is_available():
    print(f"CUDA Version: {torch.version.cuda}")
else:
    print("CUDA는 사용할 수 없습니다.")

CUDA Version: 12.4


In [4]:
import os
folder_path = './open/train/Weathered_Rock/'
total_file_count = 0

for root, dirs, files in os.walk(folder_path):
    total_file_count += len(files)

print(f"전체 파일 개수 (서브폴더 포함): {total_file_count}")

전체 파일 개수 (서브폴더 포함): 33452


In [5]:
from pathlib import Path

folder_path = Path('./open/train/Mud_Sandstone')

subdirs = [d for d in folder_path.iterdir() if d.is_dir()]

print(f"'{folder_path}' 안의 하위 폴더 목록:")
for subdir in subdirs:
    print(subdir.name)  # 폴더 이름만 출력

'open/train/Mud_Sandstone' 안의 하위 폴더 목록:


In [6]:
import timm
import torch

# 별도로 해당 모델에 맞는 입력 전처리 방법을 확인하고 적용해야 합니다.
# 위 코드의 resolve_data_config와 create_transform 부분이 바로 이 역할을 합니다. 
# timm을 사용할 때는 이 전처리 부분을 놓치지 않는 것이 매우 중요합니다.
available_swin_models = timm.list_models('*swin*', pretrained=True)
print(available_swin_models[:5])

['hiera_small_abswin_256.sbb2_e200_in12k', 'hiera_small_abswin_256.sbb2_e200_in12k_ft_in1k', 'hiera_small_abswin_256.sbb2_pd_e200_in12k', 'hiera_small_abswin_256.sbb2_pd_e200_in12k_ft_in1k', 'swin_base_patch4_window7_224.ms_in1k']


/home/metaai2/miniconda3/envs/env312_cuda124_torch260/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# 2. 특정 Swin Transformer 모델 불러오기 (ImageNet 사전 훈련 가중치 포함)
model_name = 'swin_large_patch4_window12_384.ms_in22k_ft_in1k' # 원하는 Swin 모델 이름 선택
transfer_model_swin = timm.create_model(model_name, pretrained=True)

In [8]:
transfer_model_swin

SwinTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 192, kernel_size=(4, 4), stride=(4, 4))
    (norm): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
  )
  (layers): Sequential(
    (0): SwinTransformerStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): SwinTransformerBlock(
          (norm1): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
          (attn): WindowAttention(
            (qkv): Linear(in_features=192, out_features=576, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=192, out_features=192, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (softmax): Softmax(dim=-1)
          )
          (drop_path1): Identity()
          (norm2): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=192, out_features=768, bias=True)
            (act): GELU(approximate='none')
            (

In [9]:
print(transfer_model_swin.head)

ClassifierHead(
  (global_pool): SelectAdaptivePool2d(pool_type=avg, flatten=Identity())
  (drop): Dropout(p=0.0, inplace=False)
  (fc): Linear(in_features=1536, out_features=1000, bias=True)
  (flatten): Identity()
)


In [10]:
# 3. 분류기(Classifier Head) 수정 (내 데이터셋 클래스 수에 맞게)
num_classes = 7 # 암석 종류 7가지
in_features = transfer_model_swin.head.fc.in_features # transfer_model_swin.head로 transfer_model_swin.head 인지 .fc를 붙이는 지 차이가 있다.
transfer_model_swin.head.fc = torch.nn.Linear(in_features, num_classes)  # fc 레이어 교체

In [11]:
print(transfer_model_swin.head) # 바뀐 head 구조 확인

ClassifierHead(
  (global_pool): SelectAdaptivePool2d(pool_type=avg, flatten=Identity())
  (drop): Dropout(p=0.0, inplace=False)
  (fc): Linear(in_features=1536, out_features=7, bias=True)
  (flatten): Identity()
)


In [12]:
# timm 모델은 torchvision의 weights.transforms() 와 같은 자동 전처리 기능을 제공하지 않음
# 따라서, 모델에 맞는 전처리 방법을 직접 설정해야 함
# 모델에 맞는 입력 설정값 가져오기 (이미지 크기, 정규화 평균/표준편차 등)
config = timm.data.resolve_data_config({}, model=transfer_model_swin)
config

{'input_size': (3, 384, 384),
 'interpolation': 'bicubic',
 'mean': (0.485, 0.456, 0.406),
 'std': (0.229, 0.224, 0.225),
 'crop_pct': 1.0,
 'crop_mode': 'center'}

In [13]:
# 학습용 Transform
train_transform = timm.data.transforms_factory.create_transform(**config, is_training=True)
# 검증/테스트용 Transform
val_transform = timm.data.transforms_factory.create_transform(**config, is_training=False) # 변수 이름 명확화

In [14]:
train_transform

Compose(
    RandomResizedCropAndInterpolation(size=(384, 384), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=bicubic)
    RandomHorizontalFlip(p=0.5)
    ColorJitter(brightness=(0.6, 1.4), contrast=(0.6, 1.4), saturation=(0.6, 1.4), hue=None)
    MaybeToTensor()
    Normalize(mean=tensor([0.4850, 0.4560, 0.4060]), std=tensor([0.2290, 0.2240, 0.2250]))
)

In [15]:
print(val_transform)

Compose(
    Resize(size=384, interpolation=bicubic, max_size=None, antialias=True)
    CenterCrop(size=(384, 384))
    MaybeToTensor()
    Normalize(mean=tensor([0.4850, 0.4560, 0.4060]), std=tensor([0.2290, 0.2240, 0.2250]))
)


In [18]:
from pathlib import Path 
# 데이터 경로 

train_data_path = Path("./open/train") # 실제 학습 데이터 경로
val_data_path = Path("./open/val")     # 실제 검증 데이터 경로 (이전 단계에서 생성)

In [20]:
print(val_transform)

Compose(
    Resize(size=384, interpolation=bicubic, max_size=None, antialias=True)
    CenterCrop(size=(384, 384))
    MaybeToTensor()
    Normalize(mean=tensor([0.4850, 0.4560, 0.4060]), std=tensor([0.2290, 0.2240, 0.2250]))
)


In [ ]:
from torchvision.datasets import ImageFolder
# 학습 데이터셋: train_transform 적용
train_dataset = ImageFolder(root=train_data_path, transform=train_transform)

# 검증 데이터셋: val_transform 적용
val_dataset = ImageFolder(root=val_data_path, transform=val_transform)

print(f"\n학습 데이터셋 크기: {len(train_dataset)}개 이미지")
print(f"검증 데이터셋 크기: {len(val_dataset)}개 이미지")


학습 데이터셋 크기: 342015개 이미지
검증 데이터셋 크기: 38005개 이미지


In [22]:
class_idx = train_dataset.class_to_idx
class_idx

{'Andesite': 0,
 'Basalt': 1,
 'Etc': 2,
 'Gneiss': 3,
 'Granite': 4,
 'Mud_Sandstone': 5,
 'Weathered_Rock': 6}

In [23]:
class_names = {class_idx[key]: key for key in class_idx.keys()}
class_names

{0: 'Andesite',
 1: 'Basalt',
 2: 'Etc',
 3: 'Gneiss',
 4: 'Granite',
 5: 'Mud_Sandstone',
 6: 'Weathered_Rock'}

In [24]:
from torch.utils.data import DataLoader
BATCH_SIZE = 16 # 배치 사이즈 설정 (GPU 메모리 상황에 맞게 조정)
NUM_WORKERS = 4 # 데이터 로딩을 위한 프로세스 수 (CPU 코어 수에 맞게 조정)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)
print(f"데이터 로더 생성 완료: Train 배치 {len(train_loader)}, Val 배치 {len(val_loader)}")
print(f"클래스: {train_dataset.classes}")

데이터 로더 생성 완료: Train 배치 21376, Val 배치 2376
클래스: ['Andesite', 'Basalt', 'Etc', 'Gneiss', 'Granite', 'Mud_Sandstone', 'Weathered_Rock']


In [25]:
# 모델을 GPU로 이동
DEVICE_STR = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"사용할 장치: {DEVICE_STR}")
transfer_model_swin.to(DEVICE_STR)

사용할 장치: cuda


SwinTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 192, kernel_size=(4, 4), stride=(4, 4))
    (norm): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
  )
  (layers): Sequential(
    (0): SwinTransformerStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): SwinTransformerBlock(
          (norm1): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
          (attn): WindowAttention(
            (qkv): Linear(in_features=192, out_features=576, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=192, out_features=192, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (softmax): Softmax(dim=-1)
          )
          (drop_path1): Identity()
          (norm2): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=192, out_features=768, bias=True)
            (act): GELU(approximate='none')
            (

In [26]:
import torch.nn as nn
class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2., reduction='mean'):
        super(FocalLoss, self).__init__()
        # alpha: 클래스별 가중치 (Tensor or list or float). None이면 균등 가중치.
        # gamma: Focusing 파라미터.
        # reduction: 'mean', 'sum', 'none'
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        # inputs: 모델 출력 로짓 (N, C)
        # targets: 실제 레이블 (N,)
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss) # p_t = exp(-CrossEntropy)
        focal_loss = ((1 - pt) ** self.gamma) * ce_loss

        if self.alpha is not None:
            # alpha를 target 클래스에 맞게 적용
            if isinstance(self.alpha, (float, int)):
                 alpha = torch.tensor([self.alpha] * inputs.shape[1], device=inputs.device)
            elif isinstance(self.alpha, list):
                 alpha = torch.tensor(self.alpha, device=inputs.device, dtype=torch.float32)
            elif torch.is_tensor(self.alpha):
                 alpha = self.alpha.to(device=inputs.device, dtype=torch.float32)
            else:
                 raise TypeError("alpha must be float, list or torch.Tensor")

            if alpha.shape[0] != inputs.shape[1]:
                 raise ValueError(f"alpha size {alpha.shape[0]} does not match C {inputs.shape[1]}")

            alpha_t = alpha.gather(0, targets) # 각 샘플의 정답 클래스에 해당하는 alpha 값 선택
            focal_loss = alpha_t * focal_loss

        # reduction 적용
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else: # 'none'
            return focal_loss

In [ ]:
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
# --- 손실 함수 및 옵티마이저 정의 ---
EPOCHS = 20 # 테스트용 에폭 수
NUM_CLASSES = num_classes # 암석 종류 7가지
WARMUP_EPOCHS = 5 # Warmup 에폭 수
LEARNING_RATE = 1e-5

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(transfer_model_swin.parameters(), lr=LEARNING_RATE, weight_decay=0.01)
scheduler_cosine = CosineAnnealingLR(optimizer, T_max=EPOCHS - WARMUP_EPOCHS, eta_min=1e-7)

In [28]:
print(set(train_dataset.targets))

{0, 1, 2, 3, 4, 5, 6}


In [29]:
"""
모델을 학습하고 검증하며, 검증 손실이 가장 낮은 모델 가중치를 저장합니다.

Args:
    model: 학습 및 검증할 PyTorch 모델.
    train_loader: 학습 데이터 로더.
    val_loader: 검증 데이터 로더.
    optimizer: 옵티마이저.
    criterion: 손실 함수.
    epochs: 총 학습 에폭 수.
    device: 학습에 사용할 디바이스 ('cuda' 또는 'cpu').
    num_classes: 데이터셋의 클래스 수.
    save_path: 최적 모델 가중치를 저장할 파일 경로 (Path 객체).
    scheduler: 학습률 스케줄러 (옵션).
    gradient_clipping: Gradient Clipping 임계값 (옵션).

Returns:
    dict: 학습 및 검증 손실, 정확도 기록을 담은 딕셔너리.
            {'train_losses': [], 'val_losses': [], 'accuracies': []}
"""

import torch
import torch.nn as nn
from tqdm import tqdm
import torchmetrics
from pathlib import Path # 경로 관리를 위해 추가

best_val_f1 = 0.0

def train_and_validate_best(model,
                            train_loader,
                            val_loader,
                            optimizer,
                            criterion,
                            epochs,
                            device,
                            num_classes,
                            save_path,
                            gradient_clipping=None,
                            ):

    history = {'train_losses': [], 'val_losses': [], 'accuracies': []}
    min_val_loss = float('inf')

    # 정확도 계산 메트릭 초기화 (num_classes 사용)
    accuracy_metric = torchmetrics.Accuracy(task='multiclass', num_classes=num_classes, average='macro').to(device)

    patience = 5 # 조기 종료를 위한 patience 설정
    print(f"학습 시작: 총 {epochs} 에폭")
    for epoch in range(epochs):
        # --- 학습 단계 ---
        model.train()
        running_train_loss = 0.0
        # tqdm 진행률 표시줄 설정
        train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]", leave=False)

        for images, labels in train_pbar:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # 역전파 먼저
            loss.backward()

            # --- Gradient Clipping (선택 사항) ---
            if gradient_clipping is not None:
                    total_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), gradient_clipping)

            optimizer.step()

            running_train_loss += loss.item()
            # tqdm 진행률 표시줄에 현재 손실 표시
            train_pbar.set_postfix(loss=loss.item())

        epoch_train_loss = running_train_loss / len(train_loader)
        history['train_losses'].append(epoch_train_loss)

        # --- 검증 단계 ---
        model.eval()
        running_val_loss = 0.0
        accuracy_metric.reset() # 각 에폭 시작 전 메트릭 리셋
        val_pbar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} [Val]", leave=False)

        with torch.no_grad():
            for images, labels in val_pbar:
                images, labels = images.to(device), labels.to(device)

                outputs = model(images)
                loss = criterion(outputs, labels)
                running_val_loss += loss.item()

                # 정확도 업데이트
                accuracy_metric.update(outputs, labels)
                # tqdm 진행률 표시줄에 현재 손실 표시
                val_pbar.set_postfix(loss=loss.item())

        epoch_val_loss = running_val_loss / len(val_loader)
        history['val_losses'].append(epoch_val_loss)

        # 정확도 계산
        epoch_accuracy = accuracy_metric.compute().item() # .item()으로 스칼라 값 추출
        history['accuracies'].append(epoch_accuracy)

        # 에폭 결과 출력
        print(f"Epoch [{epoch+1}/{epochs}] - "
              f"Train Loss: {epoch_train_loss:.4f}, "
              f"Val Loss: {epoch_val_loss:.4f}, "
              f"Val Accuracy: {epoch_accuracy:.4f}")

        if epoch_val_loss < min_val_loss:
            min_val_loss = epoch_val_loss
            best_epoch = epoch
            torch.save(model.state_dict(), save_path)
            print(f"  => Best model saved to {save_path} (Epoch {epoch+1}, Val Loss: {min_val_loss:.4f})")
        elif epoch - best_epoch >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

    print(f"학습 완료. 최적 모델 저장 경로: {save_path}")
    return history


In [30]:
import torch
from pathlib import Path
import timm
from tqdm import tqdm
import torch.nn.functional as F

# --- 설정 정의 ---
SAVE_MODEL_PATH = 'best_swin_large_model_test_limited_batches.pth'


save_path_obj = Path(SAVE_MODEL_PATH)

training_history = train_and_validate_best(
    transfer_model_swin,
    train_loader,
    val_loader,
    optimizer,
    criterion,
    EPOCHS,
    DEVICE_STR,
    NUM_CLASSES,
    save_path_obj,
    gradient_clipping=1.0 # 선택 사항: Gradient Clipping 임계값 설정 (None으로 설정하면 사용하지 않음)
)

print("\n--- 테스트 학습 종료 ---")
print(f"최적 모델 저장 경로 (테스트 중 저장되었다면): {save_path_obj}")

# --- (선택 사항) 학습 결과 활용 ---
if training_history['accuracies'] and not all(torch.isnan(torch.tensor(training_history['accuracies']))): # NaN 값 제외하고 확인
    print(f"마지막 에폭 검증 정확도: {training_history['accuracies'][-1]:.4f}")
    valid_val_losses = [loss for loss in training_history['val_losses'] if not torch.isnan(torch.tensor(loss))]
    if valid_val_losses:
        best_epoch_idx = training_history['val_losses'].index(min(valid_val_losses))
        print(f"최고 검증 성능 에폭: {best_epoch_idx + 1}")
        print(f"  - 검증 손실: {training_history['val_losses'][best_epoch_idx]:.4f}")
        print(f"  - 검증 정확도: {training_history['accuracies'][best_epoch_idx]:.4f}")
    else:
        print("유효한 검증 손실 기록이 없습니다.")
else:
    print("학습/검증 결과가 없습니다 (처리된 배치 없음).")

학습 시작: 총 20 에폭


KeyboardInterrupt: 

In [31]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
# from torchvision import transforms # timm의 transform을 사용하므로 주석 처리 가능
from PIL import Image
import os
import timm # timm 라이브러리 import

# --- 기본 설정 ---
BATCH_SIZE = 16
NUM_WORKERS = 4
TEST_CSV_PATH = './open/test.csv'
IMAGE_BASE_DIR = './open' # test.csv의 img_path가 상대 경로일 경우 기준점

# --- 모델 및 설정 (이 부분은 이미 로드되어 있다고 가정) ---
# 예시: transfer_model_swin = timm.create_model(...)
# transfer_model_swin = None # Placeholder - 실제 모델 객체로 대체 필요

# # 모델 로드 예시 (실제 코드에 맞게 수정)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# transfer_model_swin = timm.create_model('swin_base_patch4_window12_384', pretrained=True) # 예시 모델 이름
# # 필요하다면 체크포인트 로드
# # transfer_model_swin.load_state_dict(torch.load('path/to/your/model_weights.pth'))
# transfer_model_swin = transfer_model_swin.to(device)
# transfer_model_swin.eval() # 평가 모드 설정

# --- timm을 사용한 변환 설정 ---
# !!! 중요 !!!: 아래 transfer_model_swin 변수가 실제 로드된 모델 객체를 참조해야 합니다.
if 'transfer_model_swin' not in locals() or transfer_model_swin is None:
     raise NameError("`transfer_model_swin` is not defined. Please load your model first.")

# 1. 모델로부터 데이터 설정 확인
config = timm.data.resolve_data_config({}, model=transfer_model_swin)
print("Resolved timm data config:", config)

# 2. 설정 기반으로 테스트용 변환 생성 (is_training=False)
test_transform = timm.data.create_transform(**config, is_training=False)
print("Created test transform using timm:")
print(test_transform)


# --- 테스트 데이터용 Dataset 클래스 정의 (이전과 거의 동일) ---
class RockTestDataset(Dataset):
    def __init__(self, csv_path, img_dir, transform=None):
        """
        Args:
            csv_path (string): test.csv 파일 경로.
            img_dir (string): 이미지 파일들이 있는 기본 디렉토리 경로.
            transform (callable, optional): 샘플에 적용될 변환.
        """
        try:
            self.test_frame = pd.read_csv(csv_path)
        except FileNotFoundError:
             raise FileNotFoundError(f"Test CSV file not found at: {csv_path}")

        self.img_dir = img_dir
        self.transform = transform
        print(f"Loaded test CSV from {csv_path} with {len(self.test_frame)} samples.")

        if 'img_path' not in self.test_frame.columns:
            raise ValueError("CSV file must contain an 'img_path' column.")
        # ID 컬럼은 submission 생성 시 필요하므로 확인
        if 'ID' not in self.test_frame.columns:
            raise ValueError("CSV file must contain an 'ID' column for submission.")


    def __len__(self):
        return len(self.test_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_relative_path = self.test_frame.iloc[idx]['img_path']

        # 이미지 전체 경로 구성 (환경에 맞게 조정 필요)
        if img_relative_path.startswith('./'):
             # './open/test_imgs/...' 같은 형태 처리
             img_full_path = os.path.join(os.path.dirname(TEST_CSV_PATH), img_relative_path.lstrip('./'))
        elif os.path.isabs(img_relative_path):
             # 절대 경로인 경우
             img_full_path = img_relative_path
        else:
             # img_dir 기준 상대 경로인 경우
             img_full_path = os.path.join(self.img_dir, img_relative_path)

        try:
            # 이미지 로드 (RGB 변환 필수)
            image = Image.open(img_full_path).convert('RGB')
        except FileNotFoundError:
            print(f"ERROR: Image not found at {img_full_path} (Index: {idx})")
            # 오류 발생 시 처리 (예: 빈 텐서 반환 후 예측 시 처리 또는 예외 발생)
            raise FileNotFoundError(f"Image file not found: {img_full_path}")
        except Exception as e:
            print(f"ERROR: Could not load image {img_full_path} (Index: {idx}): {e}")
            raise RuntimeError(f"Failed to load image: {img_full_path}") from e

        # 변환 적용
        if self.transform:
            image = self.transform(image)

        # 예측 함수가 input만 필요하면 image만 반환
        # 만약 파일명 등 추가 정보가 필요하면 튜플로 반환 (예: return image, self.test_frame.iloc[idx]['ID'])
        # 이전 `predict` 함수는 (inputs, _) 형태를 가정했으므로, 아무 값이나 추가로 반환해도 됨
        img_id = self.test_frame.iloc[idx]['ID'] # ID를 함께 반환하는 예시
        return image, img_id


# --- Test Dataset 인스턴스 생성 ---
try:
    test_dataset = RockTestDataset(csv_path=TEST_CSV_PATH,
                                   img_dir=IMAGE_BASE_DIR,
                                   transform=test_transform)
except (FileNotFoundError, ValueError, NameError) as e:
    print(f"Error creating Test Dataset: {e}")
    # 적절한 오류 처리 (예: 프로그램 종료)
    exit()
except Exception as e:
    print(f"An unexpected error occurred during Test Dataset creation: {e}")
    exit()


# --- Test DataLoader 생성 ---
test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False, # 테스트 시에는 순서를 섞지 않습니다!
    num_workers=NUM_WORKERS,
    pin_memory=True # GPU 사용 시 권장
)

print(f"Test DataLoader created. Number of batches: {len(test_loader)}")

# 이제 이 test_loader를 create_submission_file 함수의 loader 인자로 사용할 수 있습니다.
# 예시:
# if __name__ == "__main__":
#     ... (모델 로드, device 설정 - 이미 되어 있다고 가정) ...
#
#     # 필수 변수 확인 (model 객체가 로드되었는지)
#     if 'transfer_model_swin' not in locals() or transfer_model_swin is None:
#         print("Error: 'transfer_model_swin' is not defined or loaded.")
#     elif test_loader is None:
#          print("Error: 'test_loader' could not be created.")
#     else:
#         # 이전 답변의 함수 사용
#         create_submission_file(
#             model=transfer_model_swin, # 실제 로드된 모델 객체
#             loader=test_loader,
#             device=device, # device 설정 필요
#             test_csv_path=TEST_CSV_PATH,
#             submission_csv_path='./submission.csv', # 저장 경로 설정
#             class_names={...} # 클래스 이름 딕셔너리 설정
#         )

Resolved timm data config: {'input_size': (3, 384, 384), 'interpolation': 'bicubic', 'mean': (0.485, 0.456, 0.406), 'std': (0.229, 0.224, 0.225), 'crop_pct': 1.0, 'crop_mode': 'center'}
Created test transform using timm:
Compose(
    Resize(size=384, interpolation=bicubic, max_size=None, antialias=True)
    CenterCrop(size=(384, 384))
    MaybeToTensor()
    Normalize(mean=tensor([0.4850, 0.4560, 0.4060]), std=tensor([0.2290, 0.2240, 0.2250]))
)
Loaded test CSV from ./open/test.csv with 95006 samples.
Test DataLoader created. Number of batches: 5938


In [30]:
# 저장된 가중치 불러오기
MODEL_NAME = 'swin_large_patch4_window12_384.ms_in22k_ft_in1k'
SAVED_MODEL_PATH = 'best_swin_large_model_test_limited_batches.pth'
NUM_CLASSES  = 7 # 암석 종류 7가지
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 동일한 모델 아키텍처 생성
transfer_model_swin = timm.create_model(MODEL_NAME, pretrained=False, num_classes=NUM_CLASSES)
saved_state_dict = torch.load(SAVED_MODEL_PATH, map_location=torch.device('cpu'))

# 모델에 가중치 적용
transfer_model_swin.load_state_dict(saved_state_dict)

# gpu로 이동
print(device)
transfer_model_swin = transfer_model_swin.to(device)
# 모델을 평가모드로 설정
transfer_model_swin.eval()

cuda


SwinTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 192, kernel_size=(4, 4), stride=(4, 4))
    (norm): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
  )
  (layers): Sequential(
    (0): SwinTransformerStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): SwinTransformerBlock(
          (norm1): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
          (attn): WindowAttention(
            (qkv): Linear(in_features=192, out_features=576, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=192, out_features=192, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (softmax): Softmax(dim=-1)
          )
          (drop_path1): Identity()
          (norm2): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=192, out_features=768, bias=True)
            (act): GELU(approximate='none')
            (

In [31]:
import pandas as pd
from tqdm import tqdm

# DataFrame에서 이미지 경로를 읽고, CustomDataset으로 데이터셋 생성
test = pd.read_csv('./open/test.csv')

# 예측 및 시각화 함수
def predict_and_visualize(model, loader, device):
    model.eval()
    preds = []
    
    with torch.no_grad():
        for inputs, filenames in tqdm(iter(loader)):
            inputs = inputs.to(device)
            outputs = model(inputs)
            pred = torch.argmax(outputs, dim=1)
            
            preds += pred.detach().cpu().numpy().tolist()

    return preds  # 예측 결과를 반환

# inference 함수와 predict_and_visualize를 결합한 함수
def inference_and_visualize(model, loader, device):
    preds = predict_and_visualize(model, loader, device)
    
    # 예측 결과를 원본 DataFrame에 추가
    test['predictions'] = preds
    
    # 예측 결과를 새로운 CSV 파일로 저장
    test.to_csv('./test_with_predictions.csv', index=False)
    print("예측 결과가 test_with_predictions.csv에 저장되었습니다.")
    
    return preds

device = "cuda"
inference_and_visualize(transfer_model_swin, test_loader, device)


100%|██████████| 5938/5938 [14:52<00:00,  6.65it/s]


예측 결과가 test_with_predictions.csv에 저장되었습니다.


[5,
 5,
 5,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 6,
 4,
 4,
 4,
 4,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 3,
 3,
 4,
 4,
 4,
 4,
 0,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 0,
 5,
 5,
 5,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 3,
 4,
 4,


In [32]:
df = pd.read_csv("./test_with_predictions.csv")
df.head()

,ID,img_path,predictions
0,TEST_00000,./test/TEST_00000.jpg,5
1,TEST_00001,./test/TEST_00001.jpg,5
2,TEST_00002,./test/TEST_00002.jpg,5
3,TEST_00003,./test/TEST_00003.jpg,4
4,TEST_00004,./test/TEST_00004.jpg,4


In [33]:
import pandas as pd

# 1. test.csv 읽기
df = pd.read_csv("./test_with_predictions.csv")

class_names = {
    0: 'Andesite',
    1: 'Basalt',
    2: 'Etc',
    3: 'Gneiss',
    4: 'Granite',
    5: 'Mud_Sandstone',
    6: 'Weathered_Rock'
}

# 2. rock_type 컬럼을 예측값으로 설정
mapped_preds = [class_names[p] for p in df["predictions"]]
df["rock_type"] = mapped_preds

# 3. 필요 없는 img_path 컬럼 제거
df = df[["ID", "rock_type"]]

# 4. 저장
df.to_csv("test_with_predictions.csv", index=False)